# Caffe的Python接口学习(8)：caffemodel中的参数及特征提取

如果用公式：$y = f(w \times x) + b$来表示整个运算过程的话，那么w和b就是我们需要训练的东西，w称为权值，在CNN中也可以叫做卷积和(filter)，b是偏置项。f是激活函数，有sigmoid、relu等。x就是输入的数据。

参数训练完成后，保存在caffemodel里面，实际上就是各层的w和b值。

我们运行代码：

``` python
# deploy文件
deploy = root + 'mnist/deploy.prototxt'
# 训练好的caffemodel
caffe_model = root + 'mnist/lenet_iter_9380.caffemodel'
# 加载model和network
net = caffe.Net(deploy, caffe_model, caffe.TEST)
```

就把所有的参数和数据都加载到一个net变量里面了，但是net是一个很复杂的object，想直接显示出来看是不行的，其中：

net.params：保存各层的参数值(w和b)

net.blobs：保存各层的数据值

可用命令：

``` python
[(k, v[0].data) for k, v in net.params.items()]
```

查看各层的参数值，其中k表示层的名称，v\[0\].data就是各层的W值，而v\[1\].data是各层的b值。注意：并不是所有层都有参数，只有卷积层和全连接层才有。

也可以不查看具体值，只想看一下shape，可用命令：

``` python
[(k, v[0].data.shape) for k, v in net.params.items()]
```

假设我们知道其中第一个卷积层的名字叫"Convolution1"，则我们可以提取这个层的参数：

``` python
w1 = net.params['Concolution1'][0].data
b1 = net.params['Convolution2'][1].data
```

输入这些代码，实际查看一下，对你理解network非常用帮助。

同理，除了看参数，我们还可以看数据，但是要注意的是，net里面刚开始是没有数据的，需要运行：

``` python
net.forward()
```
之后才会有数据，我们可以用代码：

``` python
[(k, v.data.shape) for k, v in net.blobs.items()]
```
或
``` python
[(k, v.data) for k, v in net.blobs.items()]
```
来查看各层的数据。注意和上面查看参数的区别，一个是net.params，一个是net.blobs。

实际上，数据刚输入的时候，我们叫图片数据，卷积之后我们就叫特征了。

如果要抽取第一个全连接层的特征，则可用命令：

``` python
fea = net.blobs['InnerProduct1'].data
```

只要知道某个层的名称，就可以抽取这个层的特征。

推荐大家spyder中，运行一下上面的所有代码，深入理解模型各层。

最后，总结一个代码：

``` python
# -*- coding: utf-8 -*-
import caffe
import numpy as np

# 父目录
root = '/root/Repository/caffe/examples/mine/'
# deploy文件
deploy = root + 'mnist/deploy.prototxt'
# 训练好的caffemodel
caffe_model = root + 'mnist/lenet_iter_9380.caffemodel'

# 加载model到network
net = caffe.Net(deploy, caffe_model, caffe.TEST)

# 查看各层参数规模
[(k, v[0].data.shape) for k, v in net.params.items()]
# 提取参数w
w1 = net.params['Convolution1'][0].data
# 提取参数b
b1 = net.params['Convolution1'][1].data

# 运行测试
net.forward()

# 查看各层数据规模
[(k, v.data.shape) for k, v in net.blobs.items()]
# 提取某层数据(或叫作：特征)
fea = net.blobs['InnerProduct1'].data
```